In [1]:
dataset = 'Musical_Instruments'

reviews_filepath = './data/raw_data/reviews_{0}_5.json.gz'.format(dataset)
metadata_filepath = './data/metadata/meta_{0}.json.gz'.format(dataset)

In [2]:
# %load modules/scripts/Load\ datasets.py


# In[6]:

# reviews_filepath = '../../data/raw_data/reviews_Musical_Instruments_5.json.gz'
# metadata_filepath = '../../data/metadata/meta_Musical_Instruments.json.gz'


# In[7]:

all_reviews = (spark
    .read
    .json(reviews_filepath))

all_metadata = (spark
    .read
    .json(metadata_filepath))



In [3]:
# %load modules/scripts/Summarize\ reviews.py


# # Summarize the reviews

# In[1]:

# all_reviews = (spark
#     .read
#     .json('../../data/raw_data/reviews_Musical_Instruments_5.json.gz'))


# In[2]:

from pyspark.sql.functions import col, expr, udf, trim
from pyspark.sql.types import IntegerType
import re

remove_punctuation = udf(lambda line: re.sub('[^A-Za-z\s]', '', line))
make_binary = udf(lambda rating: 0 if rating in [1, 2] else 1, IntegerType())

reviews = (all_reviews
    .na.fill({ 'reviewerName': 'Unknown' })
    .filter(col('overall').isin([1, 2, 5]))
    .withColumn('label', make_binary(col('overall')))
    .select(col('label').cast('int'), remove_punctuation('summary').alias('summary'))
    .filter(trim(col('summary')) != ''))


# ## Splitting data and balancing skewness

# In[3]:

train, test = reviews.randomSplit([.8, .2], seed=5436L)


# In[4]:

def multiply_dataset(dataset, n):
    return dataset if n <= 1 else dataset.union(multiply_dataset(dataset, n - 1))


# In[5]:

reviews_good = train.filter('label == 1')
reviews_bad = train.filter('label == 0')

reviews_bad_multiplied = multiply_dataset(reviews_bad, reviews_good.count() / reviews_bad.count())


train_reviews = reviews_bad_multiplied.union(reviews_good)


# ## Benchmark: predict by distribution

# In[6]:

accuracy = reviews_good.count() / float(train_reviews.count())
print('Always predicting 5 stars accuracy: {0}'.format(accuracy))


# ## Learning pipeline

# In[7]:

from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StopWordsRemover
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.classification import LogisticRegression

tokenizer = Tokenizer(inputCol='summary', outputCol='words')

pipeline = Pipeline(stages=[
    tokenizer, 
    StopWordsRemover(inputCol='words', outputCol='filtered_words'),
    HashingTF(inputCol='filtered_words', outputCol='rawFeatures', numFeatures=120000),
    IDF(inputCol='rawFeatures', outputCol='features'),
    LogisticRegression(regParam=.3, elasticNetParam=.01)
])


# ## Testing the model accuracy

# In[8]:

model = pipeline.fit(train_reviews)


# In[9]:

from pyspark.ml.evaluation import BinaryClassificationEvaluator

prediction = model.transform(test)
BinaryClassificationEvaluator().evaluate(prediction)


# ## Using model to extract the most predictive words

# In[10]:

from pyspark.sql.functions import explode
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType

words = (tokenizer
    .transform(reviews)
    .select(explode(col('words')).alias('summary')))

predictors = (model
    .transform(words)
    .select(col('summary').alias('word'), 'probability'))

first = udf(lambda x: x[0].item(), FloatType())
second = udf(lambda x: x[1].item(), FloatType())

predictive_words = (predictors
   .select(
       'word', 
       second(col('probability')).alias('positive'), 
       first(col('probability')).alias('negative'))
   .groupBy('word')
   .agg(
       F.max('positive').alias('positive'),
       F.max('negative').alias('negative')))

positive_predictive_words = (predictive_words
    .select(col('word').alias('positive_word'), col('positive').alias('pos_prob'))
    .sort('pos_prob', ascending=False))

negative_predictive_words = (predictive_words
    .select(col('word').alias('negative_word'), col('negative').alias('neg_prob'))
    .sort('neg_prob', ascending=False))


# In[11]:

import pandas as pd

pd.concat([
    positive_predictive_words.limit(10).toPandas(),
    negative_predictive_words.limit(10).toPandas() ],
    axis=1)



Always predicting 5 stars accuracy: 0.506650874636


,positive_word,pos_prob,negative_word,neg_prob
0,given,0.719084,limited,0.715598
1,job,0.710784,lackluster,0.708946
2,perfect,0.709105,ok,0.700700
3,ticket,0.708017,okay,0.699921
4,maybe,0.707469,fair,0.695551
5,disappointedone,0.706871,lightweightattractive,0.689687
6,supplies,0.704381,worst,0.683845
7,monkey,0.703659,destroyed,0.680854
8,bear,0.702184,disappointed,0.680082
9,awesome,0.700836,schallers,0.679959


In [4]:
# %load modules/scripts/User\ trustedness.py


# # User trustedness

# ## Loading data

# In[9]:

# all_reviews = (spark
#     .read
#     .json('../../data/raw_data/reviews_Musical_Instruments_5.json.gz'))


# ## Extracting ranking components

# In[10]:

reviews = all_reviews
reviews_per_reviewer = reviews.groupBy('reviewerID').count()


# In[31]:

from pyspark.sql.functions import col, udf, avg
from pyspark.sql.types import DoubleType

helpfulness_ratio = udf(
    lambda (useful, out_of): useful / float(out_of + 1), 
    returnType=DoubleType())

helpfulness = (reviews
  .select('reviewerID', helpfulness_ratio(col('helpful')).alias('helpfulness'))
  .groupBy('reviewerID')
  .agg(avg(col('helpfulness')).alias('helpfulness')))


# ## Computing rankings & visualizing the good and bad reviews from the most trusted users

# In[32]:

reviewers_trustedness = (helpfulness
    .join(reviews_per_reviewer, 'reviewerID')
    .select('reviewerID', (col('helpfulness') * col('count')).alias('trustedness')))


# In[ ]:

reviewers_trustedness.limit(10).toPandas()



,reviewerID,trustedness
0,A17A1KTVI3DG6U,3.047815
1,A36C867ZDP30NQ,2.000000
2,A5MC7LP0ZBO4Q,0.000000
3,A2DG65AWX5RJ4J,0.666667
4,A2IZ3ST24HSO4H,0.000000
5,ACWJDL1ZYX8RE,0.833333
6,A3AOPVQ7EZHTWA,7.267545
7,A3LOJ2QHXITCF7,0.000000
8,A2CARFAX5FNQT9,0.833333
9,AX11NOUMV8G95,1.000000


In [5]:
# %load modules/scripts/Recommender\ system.py


# ## Loading and indexing the data for training

# In[2]:

# all_reviews = (spark
#     .read
#     .json('../../data/raw_data/reviews_Musical_Instruments_5.json.gz'))


# In[4]:

from pyspark.sql.functions import col, expr, udf, trim
from pyspark.sql.types import IntegerType
import re

remove_punctuation = udf(lambda line: re.sub('[^A-Za-z\s]', '', line))
make_binary = udf(lambda rating: 0 if rating in [1, 2] else 1, IntegerType())

reviews = all_reviews.withColumn('label', make_binary(col('overall')))


# In[5]:

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

indexing_pipeline = Pipeline(stages=[
    StringIndexer(inputCol="reviewerID", outputCol="reviewerIndex"),
    StringIndexer(inputCol="asin", outputCol="asinIndex")
])

indexer = indexing_pipeline.fit(reviews)
indexed_reviews = indexer.transform(reviews)


# In[6]:

train, _, test = [ chunk.cache() for chunk in indexed_reviews.randomSplit([.6, .2, .2], seed=1800009193L) ]


# ## Balancing data

# In[7]:

def multiply_dataset(dataset, n):
    return dataset if n <= 1 else dataset.union(multiply_dataset(dataset, n - 1))

reviews_good = train.filter('label == 1')
reviews_bad = train.filter('label == 0')

reviews_bad_multiplied = multiply_dataset(reviews_bad, reviews_good.count() / reviews_bad.count())

train_reviews = reviews_bad_multiplied.union(reviews_good)


# ## Evaluator

# In[8]:

from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    predictionCol='prediction', 
    labelCol='label')


# ## Benchmark: predict by distribution

# In[9]:

from pyspark.sql.functions import lit

average_rating = (train_reviews
    .groupBy()
    .avg('label')
    .collect()[0][0])

average_rating_prediction = test.withColumn('prediction', lit(average_rating))

average_rating_evaluation = evaluator.evaluate(average_rating_prediction)

print('The RMSE of always predicting {0} stars is {1}'.format(average_rating, average_rating_evaluation))


# ## Recommender system

# In[10]:

from pyspark.ml.recommendation import ALS

als = ALS(
        maxIter=15,
        regParam=0.1,
        userCol='reviewerIndex',
        itemCol='asinIndex',
        ratingCol='label',
        rank=24,        
        seed=1800009193L)


# ## Evaluating the model

# In[14]:

recommender_system = als.fit(train_reviews)


# In[15]:

predictions = recommender_system.transform(test)


# In[16]:

evaluation = evaluator.evaluate(predictions.filter(col('prediction') != float('nan')))

print('The RMSE of the recommender system is {0}'.format(evaluation))



The RMSE of always predicting 0.509133408363 stars is 0.491705072053
The RMSE of the recommender system is 0.389991926061


## Select a product

In [19]:
reviewed_products = (all_metadata
    .join(all_reviews, 'asin')
    .filter('''
        categories is not null 
        and related is not null'''))

(reviewed_products
     .groupBy('asin')
     .count()
     .sort('count', ascending=False)
     .limit(10)
     .toPandas())

,asin,count
0,B003VWJ2K8,163
1,B0002E1G5C,143
2,B0002F7K7Y,116
3,B003VWKPHC,114
4,B0002H0A3S,93
5,B0002CZVXM,74
6,B0006NDF8A,71
7,B0009G1E0K,69
8,B0002E2KPC,68
9,B0002GLDQM,67


In [20]:
selected_product = 'B003VWJ2K8'

## Product negative words

In [22]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import explode

import pandas as pd
pd.set_option('display.max_rows', 1000)

product_words_per_reviewer = (
    Tokenizer(inputCol='summary', outputCol='words')
        .transform(all_reviews.filter(col('asin') == selected_product))
    .select('reviewerID', 'words'))

word_ranks = (product_words_per_reviewer
    .select(explode(col('words')).alias('word'))
    .distinct()
    .join(negative_predictive_words, col('word') == negative_predictive_words.negative_word)
#     .select('word', 'neg_prob')
    .sort('neg_prob', ascending=False))

word_ranks.limit(1000).toPandas()

,word,negative_word,neg_prob
0,broke,broke,0.650409
1,might,might,0.641655
2,fan,fan,0.638826
3,dead,dead,0.624869
4,pitch,pitch,0.620017
5,easier,easier,0.616248
6,brand,brand,0.600524
7,broken,broken,0.599557
8,issues,issues,0.591233
9,work,work,0.586143


In [23]:
selected_negative_word = 'broke'

## Trusted users that used the word

In [24]:
from pyspark.sql.functions import udf, lit
from pyspark.sql.types import BooleanType

is_elemen_of = udf(lambda word, words: word in words, BooleanType())

users_that_used_the_word = (product_words_per_reviewer
    .filter(is_elemen_of(lit(selected_negative_word), col('words')))
    .select('reviewerID'))

users_that_used_the_word.toPandas()

,reviewerID
0,A10VG94SAKVSC0
1,A15WZCSME5X74S


## Suggested products in the same category

In [25]:
from pyspark.sql.functions import col

product_category = (reviewed_products
    .filter(col('asin') == selected_product)
    .select('categories')
    .take(1)[0][0][0][-1])

print('Product category: {0}'.format(product_category))

Product category: Tuners


In [26]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

last_element = udf(lambda categories: categories[0][-1])

products_in_same_category = (reviewed_products
    .limit(100000)
    .filter(last_element(col('categories')) == product_category)
    .select('asin', 'title')
    .distinct())

products_in_same_category.limit(10).toPandas()

,asin,title
0,B000EEEYCW,Korg GA-40 Large Display Guitar and Bass Tuner
1,B006X735PQ,Snark SN-2 All Instrument Tuner with Tap Tempo Metronome - Includes ChromaCast Pick Sampler
2,B000EE8YPK,Korg CA-40 Large Display Auto Chromatic Tuner
3,B001XJBWXG,Korg AW2G Clip-on Chromatic Guitar Tuner
4,B004GYM7IE,Snark SN4 All Instrument Tuner Metronome
5,B0073XCXHA,Snark SN-8 Super Tight Tuner with ChromaCast Guitar Pick Sampler
6,B000M23OQ8,Qwik Tune GP1 Guitar Professor Guitar Tuner
7,B0073XCYO2,"Snark SN-5 Tuner for Guitar, Bass, Violin with ChromaCast Guitar Pick Sampler"
8,B00CDA0IUC,Guitar Tuner - Clip on - Electronic - Tune Acoustic - Electric Musical Instruments - Bass - Ukulele - Chromatic - Digital - Beginners - Pro - Good - Standard - Mini - Headstock - Easy Tuning - Accurate - Pitch - Turn 360 Degrees - Solid - Free Lesson
9,B000WGJ71U,Korg CM-100L Clip On Contact Microphone For Tuners


In [34]:
from pyspark.sql.functions import avg

indexed_products = indexer.transform(
    products_in_same_category.crossJoin(users_that_used_the_word))

alternative_products = (recommender_system
    .transform(indexed_products)
    .groupBy('asin')
    .agg(avg(col('prediction')).alias('prediction'))
    .sort('prediction', ascending=False))

alternative_products.toPandas()

,asin,prediction
0,B001E3BSKS,NaN
1,B002KDHBRU,0.614865
2,B004GYM7IE,0.598390
3,B0073XCYO2,0.576774
4,B0073XCXHA,0.572057
5,B001XJBWXG,0.571303
6,B00CDA0IUC,0.568401
7,B006X735PQ,0.548423
8,B000PR3JEM,0.546630
9,B003WZ6VVM,0.531225


In [ ]:
reviews = (all_reviews
    .filter(col('overall').isin([1, 2, 5]))
    .withColumn('label', make_binary(col('overall')))
    .select(col('label').cast('int'), remove_punctuation('summary').alias('summary'))
    .filter(trim(col('summary')) != ''))

In [ ]:
def most_contributing_summaries(product, total_reviews, ranking_model):
    reviews = total_reviews.filter(col('asin') == product).select('summary', 'overall')
    
    udf_max = udf(lambda p: max(p.tolist()), FloatType())
    
    summary_ranks = (ranking_model
        .transform(reviews)
        .select(
            'summary', 
            second(col('probability')).alias('pos_prob')))
    
    pos_summaries = { row[0]: row[1] for row in summary_ranks.sort('pos_prob', ascending=False).take(10) }
    neg_summaries = { row[0]: row[1] for row in summary_ranks.sort('pos_prob').take(10) }
    
    return pos_summaries, neg_summaries

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def present_product(product, total_reviews, ranking_model):
    pos_summaries, neg_summaries = most_contributing_summaries(product, total_reviews, ranking_model)
    
    pos_wordcloud = WordCloud(background_color='white', max_words=20).fit_words(pos_summaries)
    neg_wordcloud = WordCloud(background_color='white', max_words=20).fit_words(neg_summaries)
    
    fig = plt.figure(figsize=(15, 15))
    
    ax = fig.add_subplot(1,2,1)
    ax.set_title('Positive summaries')
    ax.imshow(pos_wordcloud, interpolation='bilinear')
    ax.axis('off')
    
    ax = fig.add_subplot(1,2,2)
    ax.set_title('Negative summaries')
    ax.imshow(neg_wordcloud, interpolation='bilinear')
    ax.axis('off')
    
    plt.show()

In [ ]:
present_product(best_product, all_reviews, full_model)